Suiside：

In [ ]:
import pandas as pd

# Load the dataset with the correct file path

csv_file_path = '/Users/cassiezhang/Desktop/DSPP/Year 1/ds/finial_project/HDPulse_data_export.csv'
# Update with your file path

# Try reading with a semicolon delimiter or adjust if needed
data_cleaned = pd.read_csv(csv_file_path, delimiter=',', skiprows=3, on_bad_lines='skip')

# Remove rows from index 70 to 93 (because Python uses 0-based indexing, 71st row is at index 70)
if data_cleaned.shape[0] > 70:
    data_cleaned = data_cleaned.drop(index=range(64, 80))  # delete 70 to 93 line 
else:
    print("if the number of data rows is less than expected, the deletion step is skipped")

# Keep only the necessary columns: 'County', 'Average Annual Count', 'Age-Adjusted Death Rate(†) - deaths per 100,000'
data_filtered = data_cleaned[['County', 'Average Annual Count', 'Age-Adjusted Death Rate(†) - deaths per 100,000']]

# Save the filtered data to a new CSV file
filtered_csv_path = '/Users/cassiezhang/Desktop/DSPP/Year 1/ds/finial_project/HDPulse_data_filtered.csv'  # Update with your desired output path
data_filtered.to_csv(filtered_csv_path, index=False)

print(f"Filtered data saved to {filtered_csv_path}")


Filtered data saved to /Users/cassiezhang/Desktop/DSPP/Year 1/ds/finial_project/HDPulse_data_filtered.csv


 forest coverage

In [ ]:
import os
import pandas as pd

# Get the script directory
current_dir = os.getcwd()

# Use relative paths to read files
file_name = "treecover_loss_by_region__ha.csv"  # Make sure the file is in the same directory as the script
file_path = os.path.join(current_dir, file_name)

# load dataset
df = pd.read_csv(file_path)

# keep column we need
columns_to_keep = ['adm2', 'umd_tree_cover_loss__year', 'umd_tree_cover_loss__ha', 'gfw_gross_emissions_co2e_all_gases__Mg']
cleaned_df = df[columns_to_keep]

# clean dataset
output_file_name = "cleaned_treecover_loss_data.csv"
output_path = os.path.join(current_dir, output_file_name)
cleaned_df.to_csv(output_path, index=False)

print(f"清理后的数据集已保存到 {output_file_name}")


清理后的数据集已保存到 cleaned_treecover_loss_data.csv


In [ ]:
import pandas as pd

# read file
tree_cover_df = pd.read_csv("cleaned_treecover_loss_data.csv")
adm2_metadata_df = pd.read_csv("adm2_metadata.csv")

#Ensure that column names are consistent for easy merging
adm2_metadata_df.rename(columns={"adm2__id": "adm2", "name": "county_name"}, inplace=True)

# Make sure the data types are consistent
tree_cover_df["adm2"] = tree_cover_df["adm2"].astype(str)
adm2_metadata_df["adm2"] = adm2_metadata_df["adm2"].astype(str)

# Merge the two data sets and match using adm2
merged_df = pd.merge(tree_cover_df, adm2_metadata_df, on="adm2", how="left")

# use county name to replace adm2
merged_df["adm2"] = merged_df["county_name"]

# only keep 2019 dataset
filtered_df = merged_df[merged_df["umd_tree_cover_loss__year"] == 2019]

# change column name
filtered_df.rename(columns={
    "umd_tree_cover_loss__ha": "tree_cover_loss_2019",
    "gfw_gross_emissions_co2e_all_gases__Mg": "gross_emissions_co2e_all_gases_Mg_2019"
}, inplace=True)

# keep relative column
final_df = filtered_df[["adm2", "umd_tree_cover_loss__year", "tree_cover_loss_2019", "gross_emissions_co2e_all_gases_Mg_2019"]]

# save data to CSV file 
output_path = "tree_cover_loss_2019_with_county_names_2019.csv"
final_df.to_csv(output_path, index=False)

print(f"2019 年数据已处理并保存到 {output_path}")


2019 年数据已处理并保存到 tree_cover_loss_2019_with_county_names_2019.csv


/var/folders/zh/q1nx6lrn0m31z31n4vtjk_fw0000gn/T/ipykernel_66144/4033669089.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={


In [ ]:
import pandas as pd

# read orginal data
input_file = "GEOINFO2023.GEOINFO-Data.csv"  
output_file = "GEOINFO2023_Cleaned.csv"  

# check CSV 
geo_data = pd.read_csv(input_file)

# check column name
print("原始列名:", geo_data.columns)

# make sure 'NAME' exist
if "NAME" in geo_data.columns:
    # delete 'NAME', New York"
    geo_data["NAME"] = geo_data["NAME"].str.replace(", New York", "", regex=False).str.strip()

    # save clean data
    geo_data.to_csv(output_file, index=False)
    print(f"清理后的数据已保存到 {output_file}")
else:
    print("错误: 数据中未找到 'NAME' 列，请检查文件格式。")


原始列名: Index(['GEO_ID', 'NAME', 'AREALAND', 'AREAWATR', 'AREALAND_SQMI',
       'AREAWATR_SQMI', 'INTPTLAT', 'INTPTLON'],
      dtype='object')
清理后的数据已保存到 GEOINFO2023_Cleaned.csv


In [ ]:
import pandas as pd

# load data
merged_data = pd.read_csv("merged_data_final.csv")
geo_data = pd.read_csv("GEOINFO2023_Cleaned.csv")

# check column name consistent
geo_data.rename(columns={
    "NAME": "County_Name",  # Match the County Name column of merged data
    "INTPTLAT": "latitude",  
    "INTPTLON": "longitude"  
}, inplace=True)

# Delete the "County" suffix to make sure the names are consistent
geo_data["County_Name"] = geo_data["County_Name"].str.replace(" County", "", regex=False)

# Ensure the data type is consistent
merged_data["County_Name"] = merged_data["County_Name"].astype(str)
geo_data["County_Name"] = geo_data["County_Name"].astype(str)

# combine data
merged_with_geo = pd.merge(
    merged_data, 
    geo_data[["County_Name", "latitude", "longitude"]], 
    on="County_Name", 
    how="left"
)

# Check if there is still missing latitude and longitude information
missing_geo = merged_with_geo[merged_with_geo["latitude"].isnull()]
if not missing_geo.empty:
    print("The following counties still lack latitude and longitude information")
    print(missing_geo["County_Name"].unique())

# save result to csv
output_path = "merged_data_with_geo_updated.csv"
merged_with_geo.to_csv(output_path, index=False)
print(f"数据已保存到 {output_path}")


数据已保存到 merged_data_with_geo_updated.csv
